In [4]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import pandas as pd

#train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('SplitV5/projection/train',target_size = (100, 100),color_mode='rgb', \
                                                 batch_size = 100,class_mode = 'binary')
test_set = test_datagen.flow_from_directory('SplitV5/projection/test',target_size = (100, 100),color_mode='rgb', \
                                            batch_size = 100,class_mode = 'binary')

checkpoint = keras.callbacks.ModelCheckpoint('AWSWeightstempP5.h5', monitor='val_acc', verbose=1, \
                                             save_best_only=False, save_weights_only=False, mode='auto', period=1)

def saveTo(acc):
    folder = "Models/projection"
    modelName = 'AWSWeightsP5-'
    ext = 'h5'
    path = '%s/%s%s.%s' % (folder, modelName, acc, ext)
    return path

def predict(modelName = 'AWSWeightstempP5.h5'):
    dataframe = pd.read_csv("TestVibsV5.csv")

    classifier = load_model(modelName)
    #classifier = load_model('AWSWeightstempR5.h5')

    rows = np.shape(dataframe)[0]
    hits = [0,0]

    for r in range(rows):
        address = dataframe['Projection'][r]
        label = dataframe['Sequences'][r]


        test_image = image.load_img(address, target_size = (100, 100),color_mode='rgb')
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        pred = np.round(result[0][0])

        tolerance = [1700, 1700]

        if (label >= tolerance[1] and pred == 1) or (label <= tolerance[0] and pred == 0):
            hits[0] += 1
        elif (label >= tolerance[1] or label <= tolerance[0]): 
            hits[1] += 1

    #print(hits)
    acc = hits[0]*100/(hits[1]+hits[0])
    print('Accuracy:', acc,'%')
    return str(acc)

def initialize():
    classifier = Sequential()

    classifier.add(Conv2D(32, (3, 3), input_shape = (100, 100, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Dropout(0.25))

    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Dropout(0.25))

    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Dropout(0.25))

    classifier.add(Flatten())
    
    #classifier.add(Dense(units = 100, activation = 'relu'))
    #classifier.add(Dense(units = 200, activation = 'relu'))
    #classifier.add(Dense(units = 400, activation = 'relu'))
    classifier.add(Dense(units = 800, activation = 'relu'))
    classifier.add(Dense(units = 400, activation = 'relu'))
    classifier.add(Dense(units = 200, activation = 'relu'))
    classifier.add(Dense(units = 100, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier

def runLoop(fresh = True, length = 10):
    for i in range(length):
        print('\nLoop ',i+1,' of ',length)
        if fresh == True and i == 0:
            classifier = initialize()
        else: 
            classifier = load_model('AWSWeightstempP5.h5')
            
        classifier.fit_generator(training_set,steps_per_epoch = 400,epochs = 1, \
                                 callbacks = [checkpoint], \
                                 validation_data = test_set,validation_steps = 200,initial_epoch=0)
        classifier.save(saveTo(predict()))
        keras.backend.clear_session()

Found 40000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [ ]:
runLoop(False, 20)


Loop  1  of  20
Epoch 1/1
400/400 [==============================] - 404s 1s/step - loss: 0.0694 - acc: 0.9739 - val_loss: 0.9681 - val_acc: 0.7846

Epoch 00001: saving model to AWSWeightstempP5.h5
Accuracy: 72.7 %

Loop  2  of  20
Epoch 1/1
400/400 [==============================] - 404s 1s/step - loss: 0.0625 - acc: 0.9765 - val_loss: 0.9995 - val_acc: 0.7784

Epoch 00001: saving model to AWSWeightstempP5.h5
Accuracy: 73.4 %

Loop  3  of  20
Epoch 1/1
400/400 [==============================] - 405s 1s/step - loss: 0.0548 - acc: 0.9796 - val_loss: 1.1069 - val_acc: 0.7788

Epoch 00001: saving model to AWSWeightstempP5.h5
Accuracy: 71.9 %

Loop  4  of  20
Epoch 1/1
400/400 [==============================] - 404s 1s/step - loss: 0.0508 - acc: 0.9805 - val_loss: 1.0191 - val_acc: 0.7771

Epoch 00001: saving model to AWSWeightstempP5.h5
Accuracy: 73.8 %

Loop  5  of  20
Epoch 1/1
400/400 [==============================] - 404s 1s/step - loss: 0.0481 - acc: 0.9833 - val_loss: 0.9473 - val